## Modify restart files

The script modifies all restart files in a directory. It applies the same modification to all restart files in the direcotry.

In [45]:
import netCDF4 as nc4
import xarray as xr
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
sys.path.append('/glade/u/home/adamhb/Earth-System-Model-Tools/')
import esm_tools
import os
pd.set_option('display.max_rows', 1500)
pd.set_option('display.max_columns', 1500)
np.set_printoptions(threshold=1500)

In [46]:
# Path to the restart files you want to modify
path_to_rest_files = '/glade/derecho/scratch/adamhb/f1870-1951_043024-1870-1951_-17e2acb6a_FATES-1449c787/run'


#/glade/derecho/scratch/adamhb/sup_043024-1951-2020_-17e2acb6a_FATES-1449c787/rest/2015-01-01-00000

do_logging = True
do_treatment = False

## Functions

In [50]:
def assign_multi_dim_variable_to_netcdf(file_path, variable_name, new_value):
    with nc4.Dataset(file_path, 'r+') as dataset:
        if variable_name in dataset.variables:
            # Access the variable
            variable = dataset.variables[variable_name]

            # Assign a value
            # The way you assign depends on the shape and dimensions of the variable
            # For a single-value variable:

            variable[...] = new_value  # Replace new_value with the value you want to assign
            #print("Changed {} to {}".format(variable_name,new_value))
            # For a multi-dimensional variable, specify indices or slices
            # Example for a 2D variable (like temperature at a specific time and place):
            # variable[time_index, place_index] = new_value

            #print(f"Value {new_value} assigned to {variable_name}.")
        else:
            print(f"Variable {variable_name} not found in the dataset.")



def find_matching_files(directory):
    """
    Finds and returns a list of file names in the specified directory that contain
    both 'clm2_' and '.r.' in their file names.

    :param directory: String, the path to the directory where the files are located
    :return: List of strings, the file names that match the criteria
    """
    matching_files = []
    for filename in os.listdir(directory):
        if "clm2_" in filename and ".r." in filename:
            matching_files.append(filename)
    return matching_files

def treat_forest(path_to_rest_file,pfts_to_treat,max_size_to_treat,cwd_scalar):

    '''
    Forest restoration treatment
    '''

    rest_file = xr.open_dataset(path_to_rest_file,decode_times=False)
    df = pd.DataFrame({'pft':rest_file.fates_pft.values,'n':rest_file.fates_nplant.values,
                       'dbh':rest_file.fates_dbh,'cwd':rest_file.fates_cwdagin_vec_001,
                       'leaf_litt':rest_file.fates_leaf_fines_vec_001})
    
    
    # Remove trees
    not_shrub = df['pft'].isin(pfts_to_treat)
    small_trees = df['dbh'] < max_size_to_treat
    df['nplant_treated'] = np.where((not_shrub) & (small_trees), 0, df['n'])

    # Remove CWD and leaf litter
    df['cwd'] = df['cwd'] * cwd_scalar
    df['leaf_litt'] = df['leaf_litt'] * cwd_scalar
    
    #print(len(df.loc[(not_shrub) & (small_trees)]),'cohorts remain')


    #Reassign
    assign_multi_dim_variable_to_netcdf(path_to_rest_file,"fates_nplant",  df['nplant_treated'].values)
    assign_multi_dim_variable_to_netcdf(path_to_rest_file,"fates_cwdagin_vec_001",  df['cwd'].values)
    assign_multi_dim_variable_to_netcdf(path_to_rest_file,"fates_leaf_fines_vec_001",  df['leaf_litt'].values)

    return df

def log_forest(path_to_rest_file):

    '''
    Forest restoration treatment
    '''

    rest_file = xr.open_dataset(path_to_rest_file,decode_times=False)
    df = pd.DataFrame({'pft':rest_file.fates_pft.values,'n':rest_file.fates_nplant.values,
                       'dbh':rest_file.fates_dbh,'cwd':rest_file.fates_cwdagin_vec_001,
                       'leaf_litt':rest_file.fates_leaf_fines_vec_001})
    
    
    pine = df['pft'] == 1.0
    cedar = df['pft'] == 2.0
    fir = df['pft'] == 3.0
    big_tree = df['dbh'] > 75
    small_tree = df['dbh'] < 45
    medium_tree = (df['dbh'] >= 45) & (df['dbh'] <= 75)

    # Log all large conifers (> 75 cm dbh)

    #print(sum((pine | cedar | fir) & big_tree))
    df['nplant_logged'] = np.where( (pine | cedar | fir) & big_tree, 0, df['n'])

    # Log 73% of medium pines
    df['nplant_logged'] = np.where(pine & medium_tree, (1.0-0.73) * df['n'], df['nplant_logged'])

    # Log 13% of medium cedars
    df['nplant_logged'] = np.where(cedar & medium_tree, (1.0-0.13) * df['n'], df['nplant_logged'])

    # Log 33% of medium firs
    df['nplant_logged'] = np.where(fir & medium_tree, (1.0-0.33) * df['n'], df['nplant_logged'])

    #Reassign
    assign_multi_dim_variable_to_netcdf(path_to_rest_file,"fates_nplant",  df['nplant_logged'].values)

    return df

In [44]:
# test_file = '/glade/derecho/scratch/adamhb/f1870-1951_043024-1870-1951_-17e2acb6a_FATES-1449c787/run/test/CZ2_equilibrium_700yrs_042524_01_-17e2acb6a_FATES-1449c787.clm2_0001.r.1620-01-01-00000.nc'
# original_file = '/glade/derecho/scratch/adamhb/f1870-1951_043024-1870-1951_-17e2acb6a_FATES-1449c787/run/CZ2_equilibrium_700yrs_042524_01_-17e2acb6a_FATES-1449c787.clm2_0001.r.1620-01-01-00000.nc'


# test_data = xr.open_dataset(test_file,decode_times=False)
# orig_data = xr.open_dataset(original_file,decode_times=False)

# pd.DataFrame('orig':orig_data.fates_nplant.values

## Apply logging

In [51]:
if do_logging == True:

    files_to_treat = sorted(find_matching_files(path_to_rest_files))
    paths_files_to_treat = [os.path.join(path_to_rest_files,f) for f in files_to_treat]
    
    for file in paths_files_to_treat:
        log_forest(file)

## Apply treatment

In [52]:
if do_treatment == True:
    files_to_treat = sorted(find_matching_files(path_to_rest_files))
    paths_files_to_treat = [os.path.join(path_to_rest_files,f) for f in files_to_treat]
    
    for file in paths_files_to_treat:
        treat_forest(path_to_rest_file = file,pfts_to_treat=[1.0,2.0,3.0,5.0],max_size_to_treat = 40.0,cwd_scalar = 0)